<a href="https://colab.research.google.com/github/ArtyomR/Q_n_A_for_Central_Bank/blob/main/Mechanics%2B1_RAG_v_0_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG для ЦБ

## Установка библиотек gigachain, chromadb

In [3]:
%pip install chromadb gigachain

## Инициализация модели Fred

In [4]:
import torch
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    GenerationConfig,
    TextStreamer,
    pipeline,
)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
generation_config = GenerationConfig.from_pretrained("Den4ikAI/FRED-T5-LARGE_text_qa")
tokenizer = AutoTokenizer.from_pretrained("Den4ikAI/FRED-T5-LARGE_text_qa")
model = AutoModelForSeq2SeqLM.from_pretrained("Den4ikAI/FRED-T5-LARGE_text_qa").to(device)
#model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Подготовка документа

Для работы с документом нам нужно разделить его на части. Для этого используем TextLoader для загрузки книги и RecursiveCharacterTextSplitter, чтобы разделить текст на приблизительно равные куски в ~500 символов с перекрытием в ~50 символов. Этот тип сплиттера сам выбирает каким способом следует оптимально разделять документ (по абзацам, по предложениям и т.д.)

## Загрудаем данные с GitHub

In [5]:
!wget https://raw.githubusercontent.com/ArtyomR/Q_n_A_for_Central_Bank/main/data_que_ans_urls_numtem_nametem.xlsx

--2024-03-14 04:05:35--  https://raw.githubusercontent.com/ArtyomR/Q_n_A_for_Central_Bank/main/data_que_ans_urls_numtem_nametem.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3074609 (2.9M) [application/octet-stream]
Saving to: ‘data_que_ans_urls_numtem_nametem.xlsx.1’

data_que_ans_urls_n 100%[===================>]   2.93M  --.-KB/s    in 0.07s   

2024-03-14 04:05:35 (40.2 MB/s) - ‘data_que_ans_urls_numtem_nametem.xlsx.1’ saved [3074609/3074609]



## Открытие датасета QnA с сайта ЦБ
В файле 5195 документов. Для тестирования рекомендуем использовать небольшое количество, например, 10.  
Укажите количество документов для загрузки в переменной DOC_QUANTITY


In [10]:
import pandas as pd

DOC_QUANTITY = 5

full_data = pd.read_excel("/content/data_que_ans_urls_numtem_nametem.xlsx", usecols = ['QUE', 'ANS', 'Urls'])
test_df = full_data[:DOC_QUANTITY]
test_df.head()

,QUE,ANS,Urls
0,Почему Банк России не хранил золотовалютные ре...,"Золотовалютные резервы — это инструмент, за сч...",http://www.cbr.ru/Reception/TopicalMessage/Pag...
1,"Можно ли было сделать что-нибудь, чтобы часть ...",Такого способа нет. Безналичная валюта всегда ...,http://www.cbr.ru/Reception/TopicalMessage/Pag...
2,"Правда ли, что Банк России вывез золото из зол...","Нет, это неправда. Все золото из наших золотов...",http://www.cbr.ru/Reception/TopicalMessage/Pag...
3,"Как иностранные ценные бумаги, приобретенные н...","Российский брокер (депозитарий), клиентом кото...",http://www.cbr.ru/Reception/TopicalMessage/Pag...
4,"Каким образом иностранные ценные бумаги, купле...",Это произошло из-за перевода клиентов к другим...,http://www.cbr.ru/Reception/TopicalMessage/Pag...


In [11]:
test_df.QUE = test_df.QUE.apply(lambda x: x.replace(u'\xa0', u' '))
test_df.ANS = test_df.ANS.apply(lambda x: x.replace(u'\xa0', u' '))
test_df.head()

<ipython-input-11-28c8cd389bd0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.QUE = test_df.QUE.apply(lambda x: x.replace(u'\xa0', u' '))
<ipython-input-11-28c8cd389bd0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.ANS = test_df.ANS.apply(lambda x: x.replace(u'\xa0', u' '))


,QUE,ANS,Urls
0,Почему Банк России не хранил золотовалютные ре...,"Золотовалютные резервы — это инструмент, за сч...",http://www.cbr.ru/Reception/TopicalMessage/Pag...
1,"Можно ли было сделать что-нибудь, чтобы часть ...",Такого способа нет. Безналичная валюта всегда ...,http://www.cbr.ru/Reception/TopicalMessage/Pag...
2,"Правда ли, что Банк России вывез золото из зол...","Нет, это неправда. Все золото из наших золотов...",http://www.cbr.ru/Reception/TopicalMessage/Pag...
3,"Как иностранные ценные бумаги, приобретенные н...","Российский брокер (депозитарий), клиентом кото...",http://www.cbr.ru/Reception/TopicalMessage/Pag...
4,"Каким образом иностранные ценные бумаги, купле...",Это произошло из-за перевода клиентов к другим...,http://www.cbr.ru/Reception/TopicalMessage/Pag...


### Разбиваем текст на чанки

In [12]:
#2я версия разбиения
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)

doc = []
for i in range(len(test_df.ANS)):
  doc.append(Document(page_content=test_df.ANS[i], metadata={"source": test_df.Urls[i]}))

print("Количество документов: ", len(doc))
doc = text_splitter.split_documents(doc)
print("Количество чанков: ", len(doc))

Количество документов:  5
Количество чанков:  13


## Создание базы данных эмбеддингов
Для работы с базой данных мы создаем объект GigaChatEmbeddings и передаем его в базу данных Chroma.

###Скачивание sentence трансформеров

In [13]:
!pip install sentence-transformers

###Эмбеддинги и создани db
На данном этапе создается класс - обертка MyEmbeddings с необходимыми функциями `embed_documents` и `embed_query` для передачи в `Chroma.from_documents`. Цель создания класса - использовать модель эмбеддингов из библиотеки `SentenceTransformer` с возможностью переноса вычислений на ГПУ

In [14]:
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer
import torch

print(torch.cuda.is_available())

class MyEmbeddings:
    def __init__(self):
        model_name='intfloat/multilingual-e5-large'
        self.model = SentenceTransformer(model_name, device = device)

    def embed_documents(self, texts):
        return self.model.encode(texts).tolist()

    def embed_query(self, texts):
        return self.model.encode(texts).tolist()


embeddings = MyEmbeddings()

db = Chroma.from_documents(
    doc,
    embeddings,
    #client_settings=Settings(anonymized_telemetry=False),
    persist_directory = "./chroma_db1"
)

False


##QnA цепочка
Делаем свою RAG-систему для работы с моделью `T5 FRED`

In [15]:
class custom_qa_chain:
  def __init__(self, model, db, generation_config, tokenizer, return_source = True):
    self.model = model
    self.generation_config = generation_config
    self.tokenizer = tokenizer
    self.db = db
    self.return_source = return_source

  def find_relevant(self, question):
    """ Функция ищет схожие с вопросом документы в векторной базе
     и единым текстом передает их как контекст"""

    list_of_source = []
    docs = self.db.similarity_search(question, k=5)
    context = '.'.join([doc.page_content for doc in docs])
    #if self.return_source:
    list_of_source = [doc.metadata for doc in docs]
    return context, list_of_source

  def make_promt(self, context, question):
    """ Функция - template для промт-сообщения, здесь в нужные места
    добавлябтся контекст и вопрос"""

    return '''<SC6>Текст: {}\nВопрос: {}\nОтвет: <extra_id_0>'''.format(context, question)

  def generate_answer(self, question):
    """ Функция генерации ответа моделью, вызывает поиск по базе,
    создает промт-сообщение и запускает генерацию ответа"""

    sources = []
    context, list_of_source = self.find_relevant(question)
    promt = self.make_promt(context, question)
    data = self.tokenizer(f"{promt}", return_tensors="pt").to(self.model.device)
    output_ids = self.model.generate(
        **data,
        generation_config=self.generation_config
    )[0]
    #if self.return_source:
    sources = list_of_source
    return self.tokenizer.decode(output_ids[2:].tolist()), context, sources

## Быстрое Тестирование

In [16]:
rag = custom_qa_chain(model, db, generation_config, tokenizer)

for question in test_df.QUE.values[:5]:
  print(f"Вопрос: {question}")
  ans, context, source = rag.generate_answer(question)

  print(f'Контекст: {context}')
  print(f"Ответ: {ans}")
  print(f"Источники: {source}")

Вопрос: Почему Банк России не хранил золотовалютные резервы внутри страны?
Контекст: Золотовалютные резервы — это инструмент, за счет которого центральный банк может защищать экономику от внешних кризисов. В тяжелых ситуациях они обеспечивают выплату валютного долга, критический импорт, стабилизируют валютный рынок. Хранить резервы внутри страны или потратить их внутри страны — это все равно что не иметь никаких резервов, вообще никакой защиты от внешних кризисов. Так наша страна жила в 1992–1999 годах. Тогда она была беззащитна перед внешними угрозами, это был практически.Золотовалютные резервы — это инструмент, за счет которого центральный банк может защищать экономику от внешних кризисов. В тяжелых ситуациях они обеспечивают выплату валютного долга, критический импорт, стабилизируют валютный рынок. Хранить резервы внутри страны или потратить их внутри страны — это все равно что не иметь никаких резервов, вообще никакой защиты от внешних кризисов. Так наша страна жила в 1992–1999 год

## Оценка качества Q&A chain при помощи SpaCy

### Готовим датасет для оценки семантической близости ответов LLM и эталонных ответов

In [17]:
from tqdm.auto import tqdm

# Ответы с сайта возьмем за ground truth, QUE переименуюм в question
validation_set = test_df[['QUE', 'ANS']].rename(columns={"QUE" : "question", 'ANS' : 'ground_truth'})

# создаем датасет для оценки ответов
def create_data_for_rag_eval(qa_chain, eval_dataset):
  rag_dataset = []
  for row in tqdm(eval_dataset.itertuples()):
    answer, context, _ = qa_chain.generate_answer(row.question)
    # Ensure contexts is a list of strings
    contexts = [] if context is None else [context]
    rag_dataset.append(
        {"question" : row.question,
         "answer" : answer,
         "contexts" : contexts,
         "ground_truths" : row.ground_truth
         }    )
  rag_df = pd.DataFrame(rag_dataset)
  #print(rag_df)
  #rag_eval_dataset = Dataset.from_pandas(rag_df)
  return rag_df

qa_rag_dataset = create_data_for_rag_eval(rag, validation_set)

0it [00:00, ?it/s]

In [18]:
qa_rag_dataset.head()

,question,answer,contexts,ground_truths
0,Почему Банк России не хранил золотовалютные ре...,"\nНет, это неправда. Все золото из наших золот...","[Золотовалютные резервы — это инструмент, за с...","Золотовалютные резервы — это инструмент, за сч..."
1,"Можно ли было сделать что-нибудь, чтобы часть ...","\nНет, это неправда. Все золото из наших золот...",[Такого способа нет. Безналичная валюта всегда...,Такого способа нет. Безналичная валюта всегда ...
2,"Правда ли, что Банк России вывез золото из зол...","\nНет, это неправда. Все золото из наших золот...","[Нет, это неправда. Все золото из наших золото...","Нет, это неправда. Все золото из наших золотов..."
3,"Как иностранные ценные бумаги, приобретенные н...","\nИностранные ценные бумаги, приобретенные на ...","[Российский брокер (депозитарий), клиентом кот...","Российский брокер (депозитарий), клиентом кото..."
4,"Каким образом иностранные ценные бумаги, купле...","\nИностранные ценные бумаги, купленные на Моск...",[Это произошло из-за перевода клиентов к други...,Это произошло из-за перевода клиентов к другим...


### Сравниваем ответ LLM и эталонные ответы (SpaCy)

In [19]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 26.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [20]:
import spacy

# Загрузка модели для русского языка
nlp = spacy.load("ru_core_news_sm")

In [21]:
def compute_similarity(answer1, answer2):
    doc1 = nlp(answer1)
    doc2 = nlp(answer2)
    similarity_score = doc1.similarity(doc2)
    return similarity_score

In [22]:
# Применяем функцию к каждой паре ответов в DataFrame
qa_rag_dataset['similarity_score'] = qa_rag_dataset.apply(lambda row: compute_similarity(row['answer'], row['ground_truths']), axis=1)

# Выводим результаты
print(qa_rag_dataset[['answer', 'ground_truths', 'similarity_score']])

<ipython-input-21-3a7909dc21e5>:4: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity_score = doc1.similarity(doc2)


                                              answer  \
0  \nНет, это неправда. Все золото из наших золот...   
1  \nНет, это неправда. Все золото из наших золот...   
2  \nНет, это неправда. Все золото из наших золот...   
3  \nИностранные ценные бумаги, приобретенные на ...   
4  \nИностранные ценные бумаги, купленные на Моск...   

                                       ground_truths  similarity_score  
0  Золотовалютные резервы — это инструмент, за сч...          0.811593  
1  Такого способа нет. Безналичная валюта всегда ...          0.465564  
2  Нет, это неправда. Все золото из наших золотов...          0.925205  
3  Российский брокер (депозитарий), клиентом кото...          0.190306  
4  Это произошло из-за перевода клиентов к другим...          0.388636  


In [23]:
# Вычислите среднее арифметическое для колонки similarity_score
average_similarity = qa_rag_dataset['similarity_score'].mean()

print(f"Среднее арифметическое значение схожести: {average_similarity:.2f}")

Среднее арифметическое значение схожести: 0.56


## История
Версия 0.4  
- удалён модуль RAGAS
- добавлен модуль проверки симантической близости на spacy
- загрузка файла с данными с GitHub

Версия 0.3  
- custom_qa_retrival.generate_answer возвращает answer, context, sources  
- добавлено заполнение атрибута source в записи БД